# Checking RFM IDs merging with vendor data
In the initial set of files from Calibr, SMILES strings appeared to match fairly well with the SMILES contained in the vendor data. However, upon their conversion to unique Reframe IDs, the SMILES strings were regenerated and do not match well with the vendor's data-- even after desalting the compounds.

## What this script looks at:
- How many unique Calibr IDs are in each file?
- How well do each of the three vendor data sets (GVK, Integrity, Informa) merge based on the internal Calibr HVAC ID, the SMILES strings, and the initial matching of the vendor data ids to the compounds.

In [55]:
import pandas as pd

In [56]:
# Read in the core files
rfm = pd.read_excel('/Users/laurahughes/Documents/repurpose/data/20180403_HVAC_info.xlsx')
rfm.head()

,internal_id,initial_source,identifier_source,smiles,high_priority,integrity_equiv_id,citeline_equiv_id,gvk_equiv_id,name,drug_name,...,matching vendor smiles,smiles from batch,library,name.1,additional_comment,internal_id2,category,status,hvac_id_can status annotation,hvac_id
0,CBR-HVAC-15267,integrity,91057,CCCC1O[C@H]2[C@](O1)(C(=O)CO)[C@@]1([C@@H](C2)...,NaN,91057.0,budesonide,5900149.0,Budesonide,BUDESONIDE,...,1.0,CCCC1O[C@H]2[C@](O1)(C(=O)CO)[C@@]1([C@@H](C2)...,(Kal) Sigma-Aldrich,BUDESONIDE,CBR-HVAC-15267: Breast Cancer-Resistant Protei...,CBR-HVAC-15267,Breast Cancer-Resistant Protein (BCRP; ABCG2) ...,CBR-HVAC-15267_assigned_and_unrevoked,CBR-HVAC-15267,NaN
1,CBR-HVAC-15267,integrity,91057,CCCC1O[C@H]2[C@](O1)(C(=O)CO)[C@@]1([C@@H](C2)...,NaN,91057.0,budesonide,5900149.0,Budesonide,BUDESONIDE,...,1.0,CCCC1O[C@H]2[C@](O1)(C(=O)CO)[C@@]1([C@@H](C2)...,(Kal) PRESTWICK,BUDESONIDE,CBR-HVAC-15267: Breast Cancer-Resistant Protei...,CBR-HVAC-15267,Breast Cancer-Resistant Protein (BCRP; ABCG2) ...,CBR-HVAC-15267_assigned_and_unrevoked,CBR-HVAC-15267,NaN
2,CBR-HVAC-15267,integrity,91057,CCCC1O[C@H]2[C@](O1)(C(=O)CO)[C@@]1([C@@H](C2)...,NaN,91057.0,budesonide,5900149.0,Budesonide,BUDESONIDE,...,1.0,CCCC1O[C@H]2[C@](O1)(C(=O)CO)[C@@]1([C@@H](C2)...,Sigma-Aldrich,Budesonide,CBR-HVAC-15267: Breast Cancer-Resistant Protei...,CBR-HVAC-15267,Breast Cancer-Resistant Protein (BCRP; ABCG2) ...,CBR-HVAC-15267_assigned_and_unrevoked,CBR-HVAC-15267,NaN
3,CBR-HVAC-04520,integrity,203367,c1([nH]c(=O)[nH]c(=O)n1)C(=O)[O-].[K+],NaN,203367.0,NaN,NaN,Oteracil potassium,NaN,...,0.0,O=c1[nH]c(=O)[nH]c(n1)C(=O)[O-],Combi Blocks,Oteracil potassium,CBR-HVAC-04520,CBR-HVAC-04520,NaN,"CBR-HVAC-04520_assigned_and_unrevoked,CBR-HVAC...",CBR-HVAC-04520,RFM-000-001-1
4,CBR-HVAC-09007,gvk,16000946,OCC(COc1ccc(cc1)Cl)O,NaN,NaN,NaN,16000946.0,NaN,CHLORPHENESIN,...,1.0,OCC(COc1ccc(cc1)Cl)O,Combi Blocks,CHLORPHENESIN,CBR-HVAC-09007,CBR-HVAC-09007,Antifungal,"CBR-HVAC-09007_assigned_and_unrevoked,CBR-HVAC...",CBR-HVAC-09007,RFM-000-003-3


In [57]:
# Checks: do all internal_id2 equal internal_id?
x = rfm.apply(lambda x: x['internal_id'] == x['internal_id2'], axis = 1)
all(x)
# Conclusion: internal_id2 is equivalent to internal_id

True

In [131]:
sum(rfm.groupby('hvac_id').calibr_id.nunique() > 1)

0

In [128]:
# Create subsets of the RFM data which only contain "matches" for the dataset
# Also drop duplicates, based on HVAC_ID, Vendor_ID, original SMILES string 
rfm_gvk = rfm[pd.notnull(rfm.gvk_equiv_id)]
rfm_gvk.drop_duplicates(subset = ['gvk_equiv_id', 'smiles', 'internal_id', 'hvac_id'], inplace = True)

rfm_int = rfm[pd.notnull(rfm.integrity_equiv_id)]
rfm_int.drop_duplicates(subset = ['integrity_equiv_id', 'smiles', 'internal_id', 'hvac_id'], inplace = True)

rfm_inf = rfm[pd.notnull(rfm.citeline_equiv_id)]
rfm_inf.drop_duplicates(subset = ['citeline_equiv_id', 'smiles', 'internal_id', 'hvac_id'], inplace = True)

/Users/laurahughes/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Users/laurahughes/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/Users/laurahughes/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [129]:
rfm_gvk.shape
rfm_int.shape

(11574, 28)

In [116]:
rfm_gvk.hvac_id.nunique()

8320

# GVK

In [59]:
# GVK source file (from late 2017)
gvk = pd.read_csv('/Users/laurahughes/Documents/repurpose/data/gvk_data_to_release.csv')
gvk.head()

,Unnamed: 0,hvac_id,gvk_id,calibr_note,drug_name,phase,drug_roa,category,mechanism,sub_smiles,synonyms,ikey
0,0,CBR-HVAC-00001,5916161,NaN,A-49816,Clinical Phase Unknown,NaN,High-Ceiling Diuretic,High-Ceiling Diuretic,Cl[H].[H]OC1=C(CN([H])[H])C=C(C=C1)C(=O)C2=CC=...,Abbott-49816,WYFYYAHANZQLGG-UHFFFAOYSA-N
1,1,CBR-HVAC-00002,5913712,NaN,ELMUSTINE,Clinical Phase Unknown,NaN,Anticancer,Anticancer,[H]OCCN([H])C(=O)N(CCCl)N=O,BRN-1956525; FFC-33; HECNU; Hecnu; NSC-294895,YJZJEQBSODVMTH-UHFFFAOYSA-N
2,2,CBR-HVAC-00003,17500746,NaN,CONTRAGESTAZOL,Preclinical,NaN,Immunosuppressant,Immunosuppressant,[H]N1N=C(N=C1C2=CC=CC(OC)=C2)C3=C(CC)C=CC=C3,DL-111; DL-111-IT; ST-1959,GXCZZAKPGMYPDJ-UHFFFAOYSA-N
3,3,CBR-HVAC-00004,5915803,NaN,CARBIZOCAINE HYDROCHLORIDE,Clinical Phase Unknown,Oral; Intravenous,Anesthetic,Anesthetic,Cl[H].[H]N(C(=O)OC(C)CN(CC)CC)C1=CC=CC=C1OCCCCCCC,BK-95; CCRIS-3899; Carbisocaine,JBWMXRJDMMPGCX-UHFFFAOYSA-N
4,4,CBR-HVAC-00005,5912196,NaN,NSC 224070,Clinical Phase Unknown,NaN,Anticancer,Anticancer,[H]OCCN([H])C1=C(N2CC2)C(=O)C(N([H])CCO[H])=C(...,BZQ; NSC-224070,MXNZCIQNSJMZST-UHFFFAOYSA-N


## 2793 compounds in GVK dataset lack Calibr IDs

In [60]:
# Remove compounds in GVK that lack Calibr IDs
gvk.shape
gvk.hvac_id.nunique()

print(str(len(gvk[pd.isnull(gvk.hvac_id)])) + " compounds without HVAC ids removed")

gvk = gvk[pd.notnull(gvk.hvac_id)]



2793 compounds without HVAC ids removed


## GVK merges
* Merging based just on Calibr IDs shows 6 compounds in the vendor data which don't have a match in the RFM dataset.
* Merging based on the GVK IDs identified in the RFM dataset is similar: 22 compounds don't match.
* Combining the GVK + Calibr IDs --> 38 (< 1%)
* Merging based on SMILES string performs more poorly: 9,434 don't join

In [61]:
# Just based on Calibr IDs
gvk1 = pd.merge(gvk, rfm, left_on = ["hvac_id"], right_on = ["internal_id"], how="left", indicator=True)
gvk1._merge.value_counts()

both          11132
left_only         6
right_only        0
Name: _merge, dtype: int64

In [62]:
# Just based on SMILES strings (original SMILES generation)
gvk2 = pd.merge(gvk, rfm, left_on = ["sub_smiles"], right_on = ["smiles"], how="left", indicator=True)
gvk2._merge.value_counts()

left_only     9434
both          1017
right_only       0
Name: _merge, dtype: int64

In [63]:
# based on second SMILES batch generation
gvk3 = pd.merge(gvk, rfm, left_on = ["sub_smiles"], right_on = ["smiles from batch"], how="left", indicator=True)
gvk3._merge.value_counts()

both          21925
left_only     10363
right_only        0
Name: _merge, dtype: int64

In [64]:
# Just based on GVK ID
gvk4 = pd.merge(gvk, rfm, left_on = ["gvk_id"], right_on = ["gvk_equiv_id"], how="left", indicator=True)
gvk4._merge.value_counts()

both          11324
left_only        22
right_only        0
Name: _merge, dtype: int64

In [65]:
# Based on GVK ID + HVAC ID
gvk4 = pd.merge(gvk, rfm, left_on = ["gvk_id", "hvac_id"], right_on = ["gvk_equiv_id", "internal_id"], how="left", indicator=True)
gvk4._merge.value_counts()

both          11097
left_only        38
right_only        0
Name: _merge, dtype: int64

In [124]:
# Full merge of all reframe data which contains a GVK ID, based on GVK ID + HVAC ID
gvk5 = pd.merge(gvk, rfm_gvk, left_on = ["gvk_id", "hvac_id"], right_on = ["gvk_equiv_id", "internal_id"], how="outer", indicator=True)
gvk5._merge.value_counts()

both          11068
right_only      893
left_only        38
Name: _merge, dtype: int64

In [125]:
893/len(rfm_gvk)
38/len(gvk)
len(gvk)

len(rfm_gvk)

11961

## GVK Summary
Calibr ID + GVK ID matches pretty well (though not perfectly) to the two datasets.  
* 893 compounds (7%) are in the RFM dataset with a GVK but NOT in the GVK vendor dataset
* 38 compounds (0.4%) of GVK vendor data which has a Calibr HVAC id don't match
* 11,068 / 11,961 match

# Integrity

In [70]:
# import integrity data
integrity = pd.read_csv('/Users/laurahughes/Documents/repurpose/data/integrity_annot_20171220.csv')
integrity.head()

,Unnamed: 0,id,smiles,name,status,int_thera_group,int_MoA,calibr_note,ikey,wikidata,PubChem CID
0,0,70002,C[C@]12C[C@@H]([C@H]3[C@H]([C@@H]1CC[C@@]2(C(=...,Prednisolone,Launched,"Muscular Dystrophy, Agents for; Asthma Therapy...",NaN,NaN,OIGNJSKKLXVSLS-VWUMJDOOSA-N,Q11426176,CID5755
1,1,70003,C[C@H]1C[C@H]2[C@@H]3CC[C@@]([C@]3(C[C@@H]([C@...,Methylprednisolone,Launched,Lymphocytic Leukemia Therapy; Antiallergy/Anti...,NaN,NaN,VHRSUDSXCMQTMA-PJHHCJLFSA-N,Q417222,CID6741
2,2,70005,Cc1ncc(n1CCO)[N+](=O)[O-],Metronidazole,Launched,Treatment of Tuberculosis; Acne Therapy; Infla...,Cytochrome P450 CYP1A2 Inhibitors,NaN,VAOCPAMSLUNLGC-UHFFFAOYSA-N,Q169569,CID4173
3,3,70008,C[C@H]1c2cccc(c2C(=O)C3=C([C@]4([C@@H]([C@H]([...,Doxycycline hydrate,Launched,Antibiotics; Female Reproductive System Cancer...,Matrix Metalloproteinase Inhibitors; Protein-A...,NaN,XQTWDDCIUJNLTR-CVHRZJFOSA-N,Q27128318,NaN
4,4,70009,c1c(c(nc(=O)[nH]1)N)F,Flucytosine,Launched,Antifungal Agents,NaN,NaN,XRECTZIEBJDKEO-UHFFFAOYSA-N,Q238490,CID3366


In [71]:
# Based on SMILES (original)
int2 = pd.merge(integrity, rfm, left_on = ["smiles"], right_on = ["smiles"], how="outer", indicator=True)
int2._merge.value_counts()

both          5640
left_only     5408
right_only       0
Name: _merge, dtype: int64

In [72]:
# Based on SMILES (original)
int3 = pd.merge(integrity, rfm, left_on = ["smiles"], right_on = ["smiles from batch"], how="outer", indicator=True)
int3._merge.value_counts()

both          3413426
left_only        9654
right_only          0
Name: _merge, dtype: int64